# Tumor Levelset Extraction

The purpose of this notebook is to create an HTML table visualizing the meshes corresponding to the levelsets of the tumor segmentation boundaries

## Step 1: Load Data

In [1]:
import nibabel as nib
import numpy as np 
import matplotlib.pyplot as plt
from skimage import measure # For marching cubes
import polyscope as ps # For mesh display
import pandas as pd
import os
import glob

def load_dictionary(metadata_path):
    df = pd.read_csv(metadata_path)
    data = {}
    for index, row in df.iterrows():
        patient_id = row["ID"]
        patient_id = "M-0".join(patient_id.split("M-")) #File paths have an extra 0 in ID
        data[patient_id] = row
    del data["UCSF-PDGM-0541"] # Skip Patient 541 because segmentation file is empty
    return data

In [2]:
def argsort(seq):
    return np.array(sorted(range(len(seq)), key=seq.__getitem__), dtype=int)

metadata_path = "Data/UCSF-PDGM-metadata_v2.csv"
all_data_path = "Data/UCSF-PDGM-v3"
data = load_dictionary(metadata_path) 

patients = list(data.keys())
diagnosis = [data[p]["Final pathologic diagnosis (WHO 2021)"] for p in patients]
dead = np.array([data[p]["1-dead 0-alive"] for p in patients])
# Sort by dead/alive first, then by diagnosis
idx = np.argsort(dead)
idx = idx[argsort([diagnosis[i] for i in idx])]
patients = [patients[i] for i in idx]


## Step 2: Extract meshes from all patients at 3 different isolevels (outer tissue, main tissue, necrotic tissue)

In [3]:
iso_levels = [0.5, 1.5, 3.5] # Isolevels to extract
for i, patient in enumerate(os.listdir(all_data_path)):
    if i%10 == 0:
        print(".", end="")
    patient_folder_path = os.path.join(all_data_path, patient)
    patient = patient[:-6]
    if patient in data:
        tumor_seg_path = patient_folder_path + "/" + patient_folder_path[-20:-6] + "_tumor_segmentation.nii.gz"
        tumor_seg_nifti = nib.load(tumor_seg_path)
        tumor_seg_mat = tumor_seg_nifti.get_fdata()
        
        #Confirm the affine matrix is indeed non-scaling + arbitrary translation vector. If not report out.
        affine_mat = tumor_seg_nifti.affine[:3, :3]
        if affine_mat[0,0] == -1 and affine_mat[1,1] == -1 and affine_mat[2,2] == 1:
            for k, level in enumerate(iso_levels):
                try:
                    data[patient]["V{}".format(k)], data[patient]["T{}".format(k)], _, _ = measure.marching_cubes(tumor_seg_mat, level)
                except:
                    # Empty mesh (only happens for necrotic tissue)
                    data[patient]["V{}".format(k)] = np.array([])
                    data[patient]["T{}".format(k)] = np.array([], dtype=int)
        else:
            print("Nonstandard scaling", patient, affine_mat)

...................................................

## Step 3: Visualize Data

In [4]:
ps.init()

center = np.array(list(tumor_seg_mat.shape))/2
camera_pos = -center

table_style = """
 table, th, td {
  border: 1px solid;
}"""

fout = open("viz/index.html", "w")
fout.write("<html><head><style>{}</style></head>\n<body>\n<table><tr><td><h2>Patient Info</h2></td>".format(table_style))
fout.write("<td><h2>Outer Tissue</h2></td><td><h2>Main Tumor</h2></td><td><h2>Necrotic Tissue</h2></td></tr>\n")
for p in patients:
    d = data[p]
    if not "V0" in d:
        print(p, "does not have mesh info")
        continue
    ps.remove_all_structures()
    meshes = []
    n_meshes = 3
    if d["V2"].size == 0:
        n_meshes = 2 # No necrotic tissue
    for k in range(n_meshes):
        meshes.append(ps.register_surface_mesh("tumor%i"%k, d["V%i"%k], d["T%i"%k]))
    
    diagnosis = d["Final pathologic diagnosis (WHO 2021)"]
    dead = ["alive", "dead"][d["1-dead 0-alive"]]
    fout.write("<tr>")
    fout.write("<td><h2>{}</h2><h2>{}</h2><h2>{}</h2>".format(p, diagnosis, dead))
    for k in range(n_meshes):
        for m in meshes:
            m.set_enabled(False)
        meshes[k].set_enabled(True)
        filename = "viz/{}_{}.jpg".format(p, k)
        ps.look_at((0, 0, 0), center)
        ps.screenshot(filename=filename)
        fout.write("<td><img src=\"{}_{}.jpg\"></td>".format(p, k))
    fout.write("</tr>\n")
    
    fout.flush()
fout.close()
    
# Downscale all images to 50%
import subprocess
subprocess.call(["mogrify", "-resize", "50%", "viz/*.jpg"])
subprocess.call(["mogrify", "-trim", "viz/*.jpg"])

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.6 (Core Profile) Mesa 22.0.5
UCSF-PDGM-0289 does not have mesh info
UCSF-PDGM-0278 does not have mesh info
UCSF-PDGM-0181 does not have mesh info
UCSF-PDGM-0175 does not have mesh info
UCSF-PDGM-0138 does not have mesh info
UCSF-PDGM-0315 does not have mesh info


0